# Full Code
To clean json weather and taxi avail files called from NEA and LTA sites <br>

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import distance
from datetime import date, timedelta
import re
import math
from collections import Counter

import copy
from tqdm import tqdm
import itertools
import joblib

import geopandas as gpd
import fiona
from shapely import geometry
from shapely import wkt
from shapely.geometry import Polygon

import warnings
warnings.filterwarnings('ignore')

## 1. Gridding Singapore
Output shape file

In [ ]:
## Getting the 4 corners of SG
TL, TR, BL, BR = np.array((103.6, 1.475)), np.array((104.05, 1.475)), np.array((103.6, 1.208)), np.array((104.05, 1.208))
length_km = np.linalg.norm(TR - TL)*100
print('length', length_km)
width_km = np.linalg.norm(TL - BL)*100
print('width', width_km)

In [ ]:
## Function to plot grids
def grid(size): # in metres
    '''
    To divide SG area into grids
    
    argument
    size: length of each grid in metres
    
    Req files
    1) SG shape file
    '''
    # Read SG map
    gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
    whole_sg = gpd.read_file('../singapore-police-force-npc-boundary/spf-boundaries.kml', driver='KML')
    
    # Divide length and widths
    vertical = round(45000/size)
    horizontal = round(26700/size)
    
    length_blocks = np.linspace(103.6, 104.05, num=vertical+1) #x
    width_blocks = np.linspace(1.208, 1.475, num=horizontal+1) #y
    
    # Plotting
    y = [1.475, 1.475, 1.208, 1.208]
    x = [103.6, 104.05, 103.6, 104.05]
    
    whole_sg.plot(figsize=(15,15))
    corner = sns.scatterplot(x=x,y=y, color = 'y')
    
    for i in length_blocks:
        sns.lineplot(x=[i,i], y=[1.208,1.475], color = 'black')

    for i in width_blocks:
        sns.lineplot(x=[103.6, 104.05], y=[i,i], color = 'black')
        
#grid(2000)
#plt.scatter(103.6267, 1.307992, c= 'r')

## Function to know start end of grids
def blocks_coord(size): # in metres
    # Divide length and widths
    vertical = round(45000/size)
    horizontal = round(26700/size)
    
    length_blocks = np.linspace(103.6, 104.05, num=vertical+1) #x
    width_blocks = np.linspace(1.475, 1.208, num=horizontal+1) #y
    
    return length_blocks, width_blocks

length_blocks, rev_width_blocks = blocks_coord(2000)

In [ ]:
## Create Dataframe
grid_df = pd.DataFrame(
    {'grid_num': [],
    'Coordinates':[]})

count = 1
for i in range(len(rev_width_blocks)-1): # fix y first, 
    for j in range(len(length_blocks)-1): # then keep looping x (going to the right)
        # Top L, Top R, Btm R, Btm L
        p1 = geometry.Point(length_blocks[j],rev_width_blocks[i])
        p2 = geometry.Point(length_blocks[j+1],rev_width_blocks[i])
        p3 = geometry.Point(length_blocks[j+1],rev_width_blocks[i+1])
        p4 = geometry.Point(length_blocks[j],rev_width_blocks[i+1])
        
        pointList = [p1,p2,p3,p4]
        poly = geometry.Polygon([[p.x, p.y] for p in pointList])
        
        grid_df = grid_df.append(pd.DataFrame(
                            {'grid_num': count,
                            'Coordinates':[str(poly)]}), ignore_index=True)
        count +=1

        
## Convert to geopandas dataframe:  grid_num|geometry
grid_df['geometry'] = grid_df.Coordinates.apply(wkt.loads)
gdf = gpd.GeoDataFrame(grid_df, geometry='geometry')
gdf.drop('Coordinates', inplace=True, axis=1)

In [ ]:
## To filter for grids that contains SG area
gdf['intersect'] = False
for squares in range(len(gdf)):
    sq = gdf['geometry'][squares]
    for plg in range(len(whole_sg)):
        poly = whole_sg['geometry'][plg]
        if poly.intersects(sq):
            gdf.loc[squares, "intersect"] = True
            break
            
filter_grids = all_grids[all_grids['intersect'] == True]

## Export shape file
filter_grids.to_file('filter_grids_2/filter_grids_2.shp')

## 2. Processing weather and taxi tabular files 
Merge the data together into 1 dataframe<br>
This is only for ONE timestamp. Timestamps should be cleaned min is either 00 or 30; sec is 00<br>
Eg 2019-01-01T12:30:00 <br>

Weather files: <br>
1. rainfall w stations value     ---   timestamp | station_id | value
2. rainfall w stations latlon    ---  timestamp | station_id | latitude | longitude
3. humidity w stations value     ---   timestamp | station_id | value
4. humidity w stations latlon    ---   timestamp | station_id | latitude | longitude
5. temperature w stations value  ---  timestamp | station_id | value
6. temperature w stations latlon --- timestamp | station_id | latitude | longitude
<br>

Taxi files: <br>
7. taxi avail json file

### Read grid file

In [ ]:
grids_file = '../filter_grids_2/filter_grids_2.shp' ## To change directory

grids = gpd.read_file(grids_file)
grids['centroid'] = grids['geometry'].apply(lambda x: x.centroid) # get grids' centroid

# convert to dataframe
grids_df = pd.DataFrame(grids)
grids_df['centroid'] = grids_df['centroid'].astype(str)
grids_df['latlon'] = grids_df['centroid'].apply(lambda x: (float(x.split(' ')[1][1:]), float(x.split(' ')[2][:-1])))

# Get unique grid_num
grid_nums = list(grids_df['grid_num'].unique())

### Load weather datasets

In [ ]:
## To change the 6 file directories! The files should all be of the same timestamp
rain_value = 'rain_v.csv'
rain_latlon = 'rain_l.csv'
temp_value = 'temp_v.csv'
temp_latlon = 'temp_l.csv'
humid_value = 'humid_v.csv'
humid_latlon = 'humid_l.csv'

df_temp = pd.read_csv(temp_value)
df_rain = pd.read_csv(rain_value)
df_humid = pd.read_csv(humid_value)

df_temp_stn = pd.read_csv(temp_latlon)
df_rain_stn = pd.read_csv(rain_latlon)
df_humid_stn = pd.read_csv(humid_latlon)

### Processing weather files

In [ ]:
# get station latlon as a tuple
df_temp_stn['latlon'] = df_temp_stn.apply(lambda x: (x.longitude, x.latitude), axis=1)
df_rain_stn['latlon'] = df_rain_stn.apply(lambda x: (x.longitude, x.latitude), axis=1)
df_humid_stn['latlon'] = df_humid_stn.apply(lambda x: (x.longitude, x.latitude), axis=1)

In [ ]:
## Function to get weather value for each grid at each timestamp. Returns a dataframe
def get_value(weather, df, df_stn, ts, grid_nums, grids_df):
    '''
    Arguments
    weather: temp/ rain/ humid
    df: dataframe that contains weather value
    df_stn: dataframe that contains stn latlon
    ts: timestamp
    grid_nums: list of unique grid numbers
    grids_df: dataframe that contains grid numbers and their centroid latlon
    '''
    value_list = []
    df_fil = df[df['timestamp'] == ts].reset_index()
    df_stn_fil = df_stn[df_stn['timestamp'] == ts].reset_index()
    
    for i in range(len(grid_nums)): # for each grid_num
        a = grids_df.iloc[i]['latlon'] # latlon of row i grid_num
        stn_id = 0
        shortest = 1000000
        for j in range(len(df_stn_fil)): # for each station
            b = df_stn_fil.iloc[j]['latlon']
            interim = distance.euclidean(a, b) # get euclidean
            if interim < shortest:
                stn_id = df_stn_fil.iloc[j]['station_id']
                shortest = interim
            else:
                pass
            
        # after getting the nearest stn_id
        value = df_fil[df_fil['station_id'] == stn_id].reset_index()['value'][0] # get value
        value_list.append(value) # append value
    
    df_interim = pd.DataFrame({'grid_num': grid_nums, 'timestamp': [ts for x in range(len(grid_nums))], 
                               f'{weather}': value_list})
    
    return df_interim


In [ ]:
ts = df_temp_stn.iloc[0]['timestamp'] ## To change if there are other ways to get the timestamp

temp_clean = get_value('temp', df_temp, df_temp_stn, ts, grid_nums, grids_df)
rain_clean = get_value('rain', df_rain, df_rain_stn, ts, grid_nums, grids_df)
humid_clean = get_value('humid', df_humid, df_humid_stn, ts, grid_nums, grids_df)

### Load taxi json file

In [ ]:
taxi_file = 'taxi_avail.joblib' # change directory and extension of file accordingly
taxi = joblib.load(taxi_file) # change reading method according to the file type

### Processing taxi file

In [ ]:
# Get list of taxi's coord
one_list = taxi['features'][0]['geometry']['coordinates']
# Convert list to corr grid num of the coord
test = [math.ceil((i[0]-103.6)/0.020454545454545583) + (13 - math.ceil((i[1] -1.208)/0.020538461538461547))*22 for i in one_list]

# getting dictionary of items
c = Counter(test)

# Getting taxi_count for relevant grid_num
df_taxicount = pd.DataFrame({'grid_num': [float(x) for x in list(c.keys())], 
                             'taxi_count': [x[1] for x in list(c.items())]})

# Get full list of grid_num as a dataframe:  grid_num | timestamp
all_grids = grids[['grid_num']]
all_grids['timestamp'] = ts


# Merge all_grids and df_taxicount
taxi_clean = pd.merge(all_grids, df_taxicount, how='left')
taxi_clean['taxi_count'] = taxi_clean['taxi_count'].fillna(0) #fill missing taxi_count = 0


## 3. Merging all the cleaned files

In [ ]:
## Merge all together
merge_df = pd.merge(humid_clean, rain_clean)
merge_df = pd.merge(merge_df, temp_clean)
merge_df = pd.merge(merge_df, taxi_clean)
merge_df

## Feature engineering

In [ ]:
merge_df['timestamp'] = merge_df['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
merge_df['hour'] = merge_df['timestamp'].apply(lambda x: x.hour)
merge_df['month'] = merge_df['timestamp'].apply(lambda x: x.month)
merge_df['day'] = merge_df['timestamp'].apply(lambda x: x.weekday())
merge_df['minute'] = merge_df['timestamp'].apply(lambda x: x.minute)

merge_df['time_30'] = merge_df['timestamp'].apply(lambda x: x + timedelta(hours=0.5))
merge_df['time_60'] = merge_df['timestamp'].apply(lambda x: x + timedelta(hours=1))

In [ ]:
## To get y_30 and y_60 targets
# for each timestamp, get 30min later
merge_df['y_30'] = np.nan
merge_df['y_60'] = np.nan

for i in tqdm(range(len(merge_df))): # for each row
    ts = merge_df.iloc[i]['time_30']
    gridnum = merge_df.iloc[i]['grid_num']
    
    merge_df.iloc[i, merge_df.columns.get_loc('y_30')] = merge_df[(merge_df['grid_num'] == gridnum) & 
                                                                       (merge_df['timestamp'] == ts)].reset_index()['taxi_count'][0]
    

for i in tqdm(range(len(merge_df))): # for each row
    ts = merge_df.iloc[i]['time_60']
    gridnum = merge_df.iloc[i]['grid_num']
    
    merge_df.iloc[i, merge_df.columns.get_loc('y_60')] = merge_df[(merge_df['grid_num'] == gridnum) & 
                                                                       (merge_df['timestamp'] == ts)].reset_index()['taxi_count'][0]
    

In [ ]:
## merge_df is the final dataset, ready to be used for EDA/ training etc